In [57]:
import pandas as pd
import numpy as np
from collections import Counter
from cytoolz import concat
import os
import re
import sys
reload(sys)
sys.setdefaultencoding('utf8')
from nltk.corpus import stopwords

def anger(l):
    x = [w for w in re.findall(r'\bliar\b|\bliars\b|\blying\b|\blies\b|\bhypocrite\b|\bhypocrites\b|\bhypocrisy\b|\bhypocritical\b|\basshole\b|\bassholes\b|\bbullshit\b|\bdisgrace\b|\bdisgraces\b|\bdisgraced\b|\bdisgraceful\b|\bstfu\b|\bdisgusting\b|\bdisgusted\b|\bdisgusts\b|\bscum\b|\binfuriate\b|\binfuriates\b|\binfuriating',l)]
    a = [w for w in re.findall(r'\bfuck you\b|\bthe fuck up\b|\bpiece of shit\b|\bgo fuck yourself\b',l)]
    b = [w for w in re.findall(r'\b(piss)\b.*off',l)]
    c = [w for w in re.findall(r'\boff\b.*\b(piss)\b',l)]
    d = [w for w in re.findall(r'\b(fuck)\b.*\boff\b',l)]
    e = [w for w in re.findall(r'\boff\b.*\b(fuck)\b',l)]
    z = []
    lists = [x,a,b,c,d,e,]
    for q in lists:
        for s in q:
            z.append(s)
    if len(z) > 0:
        return True
    else:
        return False
    
keylist = ['liar','liars','lying','lies', 
           'bullshit', 
          'disgusting', 'disgusted','disgusts',
          'fuck',
          'asshole','assholes',
          'hypocrite','hypocrites','hypocrisy','hypocritical',
          'stfu',
          'piss','pissed','off',
          'disgrace','disgraces','disgraced','disgraceful',
          'up',
          'piece','shit',
          'scum',
          'infuriate','infuriates','infuriating','infuriated',
          'go', 'yourself']
stop = stopwords.words('english') + keylist

#See note below if the resulting document contains very few words
def angerPMI(filepath):
    df = pd.read_excel(filepath)
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False]
    df = df[df['TEXT'].str.contains('HDMA')==False]
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df = df[df['TEXT'].str.contains('FALSE')==False]
    df = df[df['TEXT'].str.contains('TRUE')==False].copy()
    df['Anger'] = df['TEXT'].str.lower().apply(anger)
    dfangry = df[df['Anger']==True]
    all = pd.value_counts(list(concat(df['bow'])))
    freq = pd.DataFrame({'All':all})
    A = pd.value_counts(list(concat(dfangry['bow'])))
    freq['Angry'] = A
    freq['PMI'] = np.log2((freq['Angry']*np.sum(freq['All'])) / (freq['All']*np.sum(freq['Angry'])))
    #The limit below may need to be changed to a smaller number for small datasets or those with few angry tweets
    freq2 = freq[freq['Angry']>10]
    freq2.sort_values(by='PMI',ascending=False)
    words = [w for w in freq2.index.values if w not in stop]
    freq3 = freq2.loc[words]
    name = os.path.basename(filepath).split('.')[0] + ' anger PMI.csv'
    freq3.sort_values(by='PMI',ascending=False).to_csv(os.path.join(os.path.dirname(filepath),name))

In [58]:
angerPMI('INSERT FILEPATH')